<h1>Pento Machine Learning Engineer Challenge</h1>
<h3>Juan Andres Tabarez Santias's solution</h3>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

In [ ]:
model = models.resnet18(pretrained=True)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.fc = nn.Linear(512, 4)

In [ ]:
composed = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
data = datasets.ImageFolder(root='./pento-ssr-challenge/dogs', transform=composed)

#data_val = datasets.ImageFolder(root='path to validation folder', transform=composed)

In [ ]:
train_loader = DataLoader(dataset=data, batch_size=16)

#val_loader = DataLoader(dataset=data_val, batch_size=16)

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam([parameters for parameters in model.parameters() if parameters.requires_grad], lr=0.001)

In [ ]:
num_epochs = 10

In [ ]:
for epoch in range(num_epochs):
    for x, y in train_loader:
        model.train()
        optimizer.zero_grad()
        z = model(x)
        loss = criterion(z, y)
        loss.backward()
        optimizer.step()

    # for x_test, y_test in val_loader:
    #     model.eval()
    #     z = model(x_test)
    #     _,y_hat = torch.max(z.data, 1)
    #     correct += (yhat == y_test).sum().item()

    # accuracy = correct / len(data_val)